In [1]:
import os
import geopandas as gpd
import pandas as pd
import gc

In [2]:
%load_ext autoreload
import icebath as icebath
from icebath.core import bathy_workflow
%autoreload 2

In [3]:
import dask
from dask.distributed import Client, LocalCluster
client = Client(processes=True, n_workers=1, threads_per_worker=1, memory_limit='14GB', dashboard_address=':8787')
client

Client Scheduler: tcp://127.0.0.1:51085 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 14.00 GB


In [4]:
# set up variables to run
# inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/2m/'
# fjord = 'JI'
# metastr='_mdf'

# inpardir = '/Volumes/Keeper/bathy_from_bergs_Kane_imagery_2021/Kane-no_usable_bergs/'
inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/KaneW2W2/'
fjord="KB"
metastr="_meta"

outdir = "/Users/jessica/projects/bathymetry_from_bergs/results/"

In [5]:
dirlist = [d for d in os.listdir(inpardir) if not d.startswith('.')]

In [6]:
print(dirlist)

['W2W2_20130510_10300100218D7000_1030010022056900_2m_lsf_v030208']


In [7]:
def recursive_dir_size(path):
    size = 0

    for x in os.listdir(path):
        if not os.path.isdir(os.path.join(path,x)):
            size += os.stat(os.path.join(path,x)).st_size
        else:
            size += recursive_dir_size(os.path.join(path,x))

    # convert to GB
    size = size / (1024.0**3)
    
    return size

In [ ]:
# run the bathymetry-from-iceberg-freeboard workflow on each dem (skipping larger dirs for now)
for indir in dirlist:
#     print(indir)
    try:
#         print(recursive_dir_size(inpardir+indir))
        if recursive_dir_size(inpardir+indir) > 1.0:
            bathy_workflow.run_workflow(inpardir+indir+'/', fjord, outdir, indir[0:14] + "icebergs.gpkg", metastr=metastr, bitmask=True)
        else:
            continue
        gc.collect()
    except NotADirectoryError:
        if recursive_dir_size(inpardir+indir) < 0.75:
            bathy_workflow.run_workflow(inpardir+'/', fjord, outdir, "JIicebergs.gpkg", metastr=metastr)
            break
        else:
            continue        

/Users/jessica/projects/bathymetry_from_bergs/github/icebath/icebath/core/build_xrds.py:22: UserWarning: This function currently assumes a constant grid and EPSG for all input files
  warnings.warn("This function currently assumes a constant grid and EPSG for all input files")


W2W2_20130510_10300100218D7000_1030010022056900_2m_lsf_seg1_dem.tif
W2W2_20130510_10300100218D7000_1030010022056900_2m_lsf_seg3_dem.tif
W2W2_20130510_10300100218D7000_1030010022056900_2m_lsf_seg2_dem.tif
Please note the transform is computed assuming a coordinate reference system    where x(min) is west and y(min) is south
Note that the new file is reprojected to have the same CRS as the dataset to which it is being added.        However, if the two CRSs are compatible, the spatial properties of the new file may be added to or overwrite the ones of the existing dataset
Note that tide model and epsg are hard coded in!
They can also be provided as keywords if the wrapper function is updated to handle them
The current fjord does not have a minimum berg size entry - using a default value!
Got labeled raster of potential icebergs for an image
about to get the list of possible bergs
Please note the transform computation is very application specific (negative y coordinates) and may need gener

distributed.nanny - WARNING - Restarting worker


In [ ]:
reslist = [f for f in os.listdir(outdir) if f.endswith('.gpkg')]

In [ ]:
print(reslist)

In [ ]:
# combine the geopackages into one geodataframe
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(outdir+res) for res in reslist],
                                 ignore_index=True), crs=gpd.read_file(outdir+reslist[0]).crs)

In [ ]:
gdf

In [ ]:
gdf.filtered_draft_mad.values

In [ ]:
gdf = gdf[gdf.filtered_draft_mad<50]
ibplot.meas_vs_infer_fig(gdf, save=False)

In [ ]:
outfn = fjord + "_combined_results.gpkg"
gdf.to_file(outdir+outfn, driver="GPKG")

In [ ]:
from icebath.utils import plot as ibplot

In [ ]:
ibplot.meas_vs_infer_fig(gdf, save=False)